In [7]:
import sys

sys.path.append("../../src")

In [8]:
import numpy as np
import gaussian_overlap
import importlib
import h5py
import pandas as pd
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import copy

from scipy.optimize import minimize

importlib.reload(gaussian_overlap)
from gaussian_overlap import *
from sto_ng_matrix import *

In [9]:
df = pd.read_csv("../make_base/sto_3g.csv")
df

,exps_1,coefs_1s,exps_2,coefs_2s,coefs_2p
0,0.109987,0.445576,0.074759,0.694673,0.387976
1,0.407154,0.535362,0.229124,0.405616,0.608867
2,2.246014,0.153482,0.979606,-0.100240,0.158557


In [10]:
zeta_1s = 2.69
zeta_2sp = 0.75
param1s = {
    "center": np.array([0, 0, 0]),
    "lmn": (0, 0, 0),
    "exps": np.array(df["exps_1"]) * zeta_1s**2,
    "coefs": np.array(df["coefs_1s"]),
}
param2s = {
    "center": np.array([0, 0, 0]),
    "lmn": (0, 0, 0),
    "exps": np.array(df["exps_2"]) * zeta_2sp**2,
    "coefs": np.array(df["coefs_2s"]),
}
param2px = {
    "center": np.array([0, 0, 0]),
    "lmn": (1, 0, 0),
    "exps": np.array(df["exps_2"]) * zeta_2sp**2,
    "coefs": np.array(df["coefs_2p"]),
}
param2py = {
    "center": np.array([0, 0, 0]),
    "lmn": (0, 1, 0),
    "exps": np.array(df["exps_2"]) * zeta_2sp**2,
    "coefs": np.array(df["coefs_2p"]),
}
param2pz = {
    "center": np.array([0, 0, 0]),
    "lmn": (0, 0, 1),
    "exps": np.array(df["exps_2"]) * zeta_2sp**2,
    "coefs": np.array(df["coefs_2p"]),
}
base1s = BasisFunction(**param1s)
base2s = BasisFunction(**param2s)
base2px = BasisFunction(**param2px)
base2py = BasisFunction(**param2py)
base2pz = BasisFunction(**param2pz)
basis = [base1s, base2s, base2px, base2py, base2pz]
Tmat = get_Tmat(basis)
Vmat = -3 * get_Vmat(basis, np.array([0, 0, 0]))
ERImat = get_ERImat(basis)
Smat = get_Smat(basis)

In [11]:
hmat = Tmat + Vmat
n_up = 2
n_dn = 1
c_up = np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0]])
# c_up = np.array([[1, 0, 0, 0, 0]])
c_dn = np.array([[1, 0, 0, 0, 0]])
new_energy = 0
for i in range(10000):
    Jmat_up = sum(
        np.einsum("ijkl,k, l->ij", ERImat, c_up[i], c_up[i]) for i in range(n_up)
    ) + sum(np.einsum("ijkl,k, l->ij", ERImat, c_dn[i], c_dn[i]) for i in range(n_dn))
    Kmat_up = sum(
        np.einsum("ijkl,j, k->il", ERImat, c_up[i], c_up[i]) for i in range(n_up)
    )
    Fmat_up = hmat + Jmat_up - Kmat_up
    Jmat_dn = sum(
        np.einsum("ijkl,k, l->ij", ERImat, c_up[i], c_up[i]) for i in range(n_up)
    ) + sum(np.einsum("ijkl,k, l->ij", ERImat, c_dn[i], c_dn[i]) for i in range(n_dn))
    Kmat_dn = sum(
        np.einsum("ijkl,j, k->il", ERImat, c_dn[i], c_dn[i]) for i in range(n_dn)
    )
    Fmat_dn = hmat + Jmat_dn - Kmat_dn
    val_up, vec_up = eigh(Fmat_up, Smat)
    val_dn, vec_dn = eigh(Fmat_dn, Smat)
    c_up = 0.9 * c_up + 0.1 * vec_up[:, :n_up].T
    c_dn = 0.9 * c_dn + 0.1 * vec_dn[:, :n_dn].T
    old_energy = new_energy
    new_energy = (
        sum(val_up[:n_up])
        + sum(val_dn[:n_dn])
        - 0.5
        * sum(
            np.einsum("ij, i, j", (Jmat_up - Kmat_up), c_up[i], c_up[i])
            for i in range(n_up)
        )
        - 0.5
        * sum(
            np.einsum("ij, i, j", (Jmat_dn - Kmat_dn), c_dn[i], c_dn[i])
            for i in range(n_dn)
        )
    )
    if abs(new_energy - old_energy) < 1e-10:
        break

In [12]:
print(c_up)
print(c_dn)
print(new_energy)
print(val_up)
print(val_dn)

[[-0.99370152 -0.02715819  0.          0.          0.        ]
 [ 0.25085589 -1.02451638  0.          0.          0.        ]]
[[-0.99277956 -0.03089636  0.          0.          0.        ]]
-7.323591052201479
[-2.3873879  -0.1877699   0.10563892  0.10563892  0.10563892]
[-2.36157061  0.07792582  0.16335958  0.16335958  0.16335958]
